In [ ]:
!pip install -U -q tensorflow_datasets

In [ ]:
import tensorflow_datasets as tfds
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
seed = 42

def get_data(lang, minlen=150, splitjoin=False):
    ds = tfds.load(f'wikipedia/20201201.{lang}', split='train', shuffle_files=False)
    texts = []
    titles = []
    for item in tqdm(ds.take(len(ds))):
        text = item['text'].numpy().decode('utf-8')
        if len(text) < minlen:
            continue
        texts.append(text)
        titles.append(item['title'].numpy().decode('utf-8'))
    if splitjoin:
        texts = [' '.join(x.split()) for x in texts]
    print(lang, len(texts))
    return texts, titles

In [ ]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
train['len'] = train['context'].apply(len)
minlen = train['len'].min()
print('minlen', minlen)

data = {'text': [], 'title': [], 'language': []}
for lang in ['hi', 'ta']:
    texts, titles = get_data(lang, minlen=minlen)
    data['text'].extend(texts)
    data['title'].extend(titles)
    data['language'].extend([lang]*len(texts))
df = pd.DataFrame(data)
df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
df

In [ ]:
df.to_csv('hita-wiki-0920.csv', index=False)